# Baseline SCA Measurements using GND and IO Pins

The following script was used to capture the baseline measurements, first for T-Test, and then for CPA.

The firmware is also built in this notebook to keep a constant binary image for later comparison.

Note both sync & async captures leave the clock output enabled. This could be turned off as it increases noise a bit, but to leave the closest comparison between the two we've used the same binaries.

If you just want to see the analysis, see Notebook TODO: Add XREF to Analysis Notebook

In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "d:/data_store"

## Firmware Builds, including Clock Input

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EXTERNAL_CLOCK -DPLL_120MHZ -DPCLK_ENABLED"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-120mhz-clkout-clkin.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EXTERNAL_CLOCK -DPLL_120MHZ"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-120mhz-clkin.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EXTERNAL_CLOCK -DPLL_15MHZ -DPCLK_ENABLED"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-15mhz-clkout-clkin.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EXTERNAL_CLOCK -DPLL_120MHZ -DPCLK_PIN_LOW"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-120mhz-clkin-pclklow.hex

In [ ]:
C_EXTRA_DEFS="-DUSE_PLL -DUSE_EXTERNAL_CLOCK -DPLL_15MHZ"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER" "$C_EXTRA_DEFS"
# compile firmware
cd ../colin-hacktest1/firmware/simpleserial-aes-spitest
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 C_EXTRA_DEFS="$4" -j
cp simpleserial-aes-CW308_SAM4S.hex  ../../../sam4s-comparisons/firmwares/fw-15mhz-clkin.hex

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 10000

Make a live plot to monitor captures

In [ ]:
splot = cw.StreamPlot()
splot.plot()

## T-Test Captures

In [ ]:
def tcapture(prefix, syncstr, fmcu, msps):
    N = 10000
    group1, group2 = capture_ttest(N, picoscope=False, splot=splot)
    nptsave(prefix + "{:d}mhz_{:s}_{:d}msps_ttest_10k".format(fmcu, syncstr, msps), group1, group2)

## Clock Input & Output Test

In [ ]:
# Set 12 MHz output to MCU, then sample at 10x that. MCU will run at frequency programmed in (120 MHz in these tests)
scope.clock.clkgen_freq = 12E6
scope.clock.adc_mul = 10

In [ ]:
#Turn on clock output
scope.io.hs2 = "clkgen"

In [ ]:
# Baud is lower as built from same source
target.baud = 38400

In [ ]:
#10K samples is enough to capture a few rounds
scope.adc.samples = 10000

In [ ]:
scope.clock

### GND Connected Test

In [ ]:
#Crank the gain WAY up for this test
scope.gain.mode = "high"
scope.gain.gain = 30

In [ ]:
# program firmware onto target,
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkout-clkin.hex") #clkout
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
# Leave jumpers JP4 and JP5 both mounted for this test
tcapture("gnd_clkin_clkout_", "sync", 120, 120)

In [ ]:
# Remove jumper JP5 and then run this block, this means the clock output won't go across the 20-pin cable, so less noise
tcapture("gnd_clkin_clkout_nojumper_", "sync", 120, 120)

In [ ]:
# program firmware onto target, takes clock input, doesn't output anything
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkin.hex") #clkout
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
# Jumper JP5 removed still (no signal across it however)
tcapture("gnd_clkin_", "sync", 120, 120)

We also capture a SHUNT clkin_clkout for reference

In [ ]:
#Gain back to normalish
scope.gain.mode = "low"
scope.gain.gain = 45

In [ ]:
tcapture("shunt_clkin_clkout_", "sync", 120, 120)

### I/O Pin High/Low Test

Leave jumpers JP4 mounted, remove JP5, connect SMA to pins CLKout & GND of breakout


In [ ]:
# program firmware onto target, takes clock input, doesn't output anything
# F/W will set the clock pin as an output & set it "high" if PCLK is disabled
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkin.hex") #clkout
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
#Crank the gain up for this test, not as high as straight GND though
scope.gain.mode = "low"
scope.gain.gain = 30

In [ ]:
tcapture("iohigh_clkin_", "sync", 120, 120)

In [ ]:
# program firmware onto target, takes clock input, doesn't output anything
# F/W will set the clock pin as an output & set it "low" with this define
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkin-pclklow.hex") #clkout
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
tcapture("iolow_clkin_", "sync", 120, 120)

## I/O Pin Low - Through Isolator

In [ ]:
# program firmware onto target, takes clock input, doesn't output anything
# F/W will set the clock pin as an output & set it "low" with this define
cw.program_target(scope, prog, "firmwares/fw-120mhz-clkin-pclklow.hex") #clkout
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

In [ ]:
#Crank the gain up for this test, not as high as straight GND though as seemed like it would clip
scope.gain.mode = "low"
scope.gain.gain = 35

In [ ]:
tcapture("icisolator_iolow_clkin_", "sync", 120, 120)

Using the optical isolator boards:
* On CW313 Target, mount both HS1 and HS2 jumpers
* On Isolator-Target, ensure HS2 connects to 50MB, HS1 connects to 50MB
* Connect both HS1 and HS2 with isolator cables
* On Isolator-ChipWhisperer, Connect HS2 to 50MB (don't connect to Mixer-RF)
* Connect 50MB to Mixer-LO (or use a jumper wire to connect 50MB to SMA, this routes it through the mixer, but should roughly work as well)

In [ ]:
tcapture("opticalisolator_extsma_iolow_clkin_", "sync", 120, 120)